In [1]:
import os
import subprocess
import cv2                                  #pip install opencv-python
import numpy as np

from PIL import Image
from scipy.fftpack import dct, idct
from pywt import dwt, idwt                  #pip install PyWavelets on Terminal

RGB - YUV Translator

In [2]:
class Translator:

    def RGBtoYUV(RGB_Color):
        R, G, B = (RGB_Color[i] for i in range(3))
        Y = 0.257 * R + 0.504 * G + 0.098 * B + 16
        U = -0.148 * R - 0.291 * G + 0.439 * B + 128
        V = 0.439 * R - 0.368 * G - 0.071 * B + 128
        return Y, U, V
    
    def YUVtoRGB(YUV_Color):
        Y, U, V = (YUV_Color[i] for i in range(3))
        B = 1.164 * (Y - 16) + 2.018 * (U - 128)
        G = 1.164 * (Y - 16) - 0.813 * (V - 128) - 0.391 * (U - 128)
        R = 1.164 * (Y - 16) + 1.596 * (V - 128)
        return R, G, B

A = [255, 0, 100]

B = Translator.RGBtoYUV(A)

print("Converted RGB {} to YUV {}".format(A, B))

A = Translator.YUVtoRGB(B)

print("Converted YUV {} to RGB {}".format(B, A))

Converted RGB [255, 0, 100] to YUV (91.335, 134.16, 232.84500000000003)
Converted YUV (91.335, 134.16, 232.84500000000003) to RGB (255.02256000000006, 0.04239499999998042, 100.12081999999998)


Resize using ffmpeg

In [ ]:
#ADDING PATH TO LOCATE INPUT IMAGE AND CREATE OUTPUT IMAGE

#image_location = "C:/Video-Coding"
#image_name = "Gato.jpg"

path = input("Paste the (absolute) path of the image you want to resize to SD:")
os.chdir(path)

image_name = input("Type the name of the image (ex: IMG_1490.jpg)")

#VERTICAL (480X640) OR HORIZONTAL (640x480) ORIENTATION

orientation = input("Type h if the image is horizontal, v if it is vertical:")

if orientation == "v":
    command = ['ffmpeg', '-y', '-i', image_name, '-vf', 'scale=480:640', 'output_image_480x640.jpg']
else:
    command = ['ffmpeg', '-y', '-i', image_name, '-vf', 'scale=640:480', 'output_image_640x480.jpg']

#EXECUTE THE FFMPEG COMMAND 

subprocess.run(command)
print("Resized image saved at", path)

Resized image saved at C:/Video-Coding


Serpentine reader

In [17]:
def serpentine(matrix):
    height, width = matrix.shape
    result = []

    # Iterate over diagonals in the matrix
    for diag in range(height + width - 1):
        if diag % 2 == 0:
            # GOING UP
            row = min(diag, height - 1)
            col = diag - row
            while row >= 0 and col < width:
                result.append(matrix[row, col])
                row -= 1
                col += 1
        else:
            # GOING DOWN
            col = min(diag, width - 1)
            row = diag - col
            while col >= 0 and row < height:
                result.append(matrix[row, col])
                row += 1
                col -= 1

    return result

In [22]:
#image_path = "C:/Video-Coding/Gato.jpg"

image_path = input("Paste the (absolute) path of the image:")

image = Image.open(image_path).convert('L')  # Turn into B&W
pixels = np.array(image)  # Convert image to numpy array

final_array = serpentine(pixels)

print(final_array[:20])

print("Visual example:\n")

mat = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

print("Original Pixels:\n")
print(mat)

# Apply zigzag traversal
zigzag_result = serpentine(mat)
print("\nSerpentine-Read Final Array:", zigzag_result)


[207, 207, 207, 207, 207, 206, 206, 206, 207, 207, 207, 207, 206, 206, 207, 208, 207, 206, 206, 207]
Visual example:

Original Pixels:

[[1 2 3]
 [4 5 6]
 [7 8 9]]

Serpentine-Read Final Array: [1, 2, 4, 7, 5, 3, 6, 8, 9]


Transform image into B&W using ffmpeg

In [ ]:
#ADDING PATH TO LOCATE INPUT IMAGE AND CREATE OUTPUT IMAGE

path = input("Paste the (absolute) path where the image you want to transform to B/W is located:")

os.chdir(path)

image_name = input("Type the name of the image (ex: IMG_1490.jpg)")

#QUALITY/COMPRESSION RATE RANGING BETWEEN [1,31], (1: HIGH QUALITY LOW COMPRESSION, 31: LOW QUALITY HIGH COMPRESSION)

quality_compression_rate = "28"  #CHANGE AS YOU WISH

command = ["ffmpeg", "-y", "-i", image_name, "-vf", "format=gray", "-q:v", quality_compression_rate, "output_gray_compressed.jpg"]

subprocess.run(command)
print("Transformed image saved at", path)

Transformed image saved at C:/Video-Coding


Run-length Encoder

In [5]:
example = ["00000001", "00000100", "00000010", "00000011", "00000100", "00000001", "00000101", "00000101", "00000001", "00000010",
 "00000101", "00000010", "00000011", "00000010", "00000010", "00000001", "00000100", "00000001", "00000101", "00000101",
 "00000101", "00000001", "00000111", "00000001", "00000011", "00000111", "00000010", "00000010", "00000110", "00000101",
 "00000111", "00000010", "00000011", "00000111", "00000101", "00000110", "00000110", "00000100", "00000111", "00000110",
 "00000111", "00000001", "00000100", "00000101", "00000010", "00000100", "00000101", "00000010", "00000111", "00000100"]


rle_encoding = []   #FINAL RLE LIST 
rle_encoding_decimal = []  #FINAL RLE (IN DECIMAL) LIST

for i in range(0,len(example)):

    if example[i] not in rle_encoding:  #IGNORE THE BIT SEQUENCES ALREADY ENCODED (AVOID REPES)
        current = example[i]
        repetitions = 0

        for j in range(0,len(example)):
            if example[j] == current:
                repetitions = repetitions + 1  #COUNTING HOW MANY TIMES A BIT SEQUENCE APPEARS

        rle_encoding.append(repetitions)  #ADD NUMBER OF REPETITIONS OF CURRENT BIT SEQUENCE
        rle_encoding.append(example[i])   #ADD CURRENT BIT SEQUENCE

print("RLE final list (binary representation):", rle_encoding)

rle_encoding_decimal = rle_encoding.copy()  #COPY FINAL RLE 

for k in range(0,len(rle_encoding_decimal)):

    if isinstance(rle_encoding_decimal[k], str):  #IF CURRENT ELEMENT IS A BIT SEQUENCE
        rle_encoding_decimal[k] = int(rle_encoding_decimal[k],2) #TRANSFORM BIT SEQUENCE TO DECIMAL REPRESENTATION

print("RLE final list (decimal representation):", rle_encoding_decimal)

RLE final list (binary representation): [8, '00000001', 7, '00000100', 10, '00000010', 4, '00000011', 10, '00000101', 7, '00000111', 4, '00000110']
RLE final list (decimal representation): [8, 1, 7, 4, 10, 2, 4, 3, 10, 5, 7, 7, 4, 6]


DCT Encoding & Decoding

In [ ]:
#image_path = "C:/Video-Coding/Gato.jpg"

image_path = input("Paste the (absolute) path of the image:")

image = cv2.imread(image_path) #Example with an image

Transformed_to_DCT = dct(image,type=2)  #Type 2 of DCT, implemented directly from scipy library

Inverse_DCT = idct(Transformed_to_DCT,type=2)

#DOCUMENTATION: https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.dct.html

DWT Encoding & Decoding

In [ ]:
#image_path = 'C:/Users/alexf/Downloads/IMG_1780.jpeg' 
#image_path = "C:/Video-Coding/Gato.jpg"

image_path = input("Paste the (absolute) path of the image:")

image = cv2.imread(image_path) #Example with an image

#whatever_input

Transformed_to_DWT = dwt(image,'db1') # implemented directly from PyWavelets library, db1 mode used to compress images

Inverse_DWT = idwt(None,Transformed_to_DWT,'db1')

#DOCUMENTATION: https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html